<a href="https://colab.research.google.com/github/KIMNAMHYEON-Kpass/love-letter-lab/blob/main/notebooks/02_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 설치/임포트/경로 준비

In [ ]:
# (필요시) 경량 설치
!pip install -q numpy pandas

import math, json, re
from pathlib import Path
import numpy as np
import pandas as pd

DATA_DIR = Path("data")
CONFIG_DIR = Path("config")
DATA_DIR.mkdir(parents=True, exist_ok=True)
CONFIG_DIR.mkdir(parents=True, exist_ok=True)

print({"data_dir": str(DATA_DIR), "config_dir": str(CONFIG_DIR)})


# 퍼스널리티 프리셋/규칙 로딩(없으면 기본 생성)

In [ ]:
# 기본 프리셋
default_presets = {
    "정중·낭만": {
        "humor":0.2, "directness":0.3, "formality":0.9,
        "emotion_intensity":0.7, "metaphor":0.8, "sincerity":0.9
    },
    "직설·건조": {
        "humor":0.1, "directness":0.9, "formality":0.6,
        "emotion_intensity":0.3, "metaphor":0.2, "sincerity":0.7
    },
    "재치·가벼움": {
        "humor":0.8, "directness":0.5, "formality":0.4,
        "emotion_intensity":0.6, "metaphor":0.6, "sincerity":0.5
    }
}

# 기본 규칙(금칙/긍·부정 어휘/비유·유머 키워드)
rules = {
    "profanity": ["씨발","좆","병신","꺼져","등신","죽어","지랄","sex","섹스","야한","폭력","협박"],
    "positive": ["좋","고맙","따뜻","응원","함께","설렘","안심","행복","사랑","배려","고마움","기쁨"],
    "negative": ["미안","불안","걱정","슬픔","후회","외로움","싫","두려움","불편","짜증"],
    "metaphor": ["별","달","계절","바람","파도","편지","햇살","꽃"],
    "humor": ["농담","웃음","장난","살짝","가볍게"]
}

# 파일로부터 로딩 시도(없으면 기본 저장)
ppath = CONFIG_DIR/"personality_presets.json"
rpath = CONFIG_DIR/"reason_rules.json"

if ppath.exists():
    personality_presets = json.loads(ppath.read_text(encoding="utf-8"))
else:
    ppath.write_text(json.dumps(default_presets, ensure_ascii=False, indent=2), encoding="utf-8")
    personality_presets = default_presets

if rpath.exists():
    reason_rules = json.loads(rpath.read_text(encoding="utf-8"))
else:
    rpath.write_text(json.dumps(rules, ensure_ascii=False, indent=2), encoding="utf-8")
    reason_rules = rules

print({"presets": list(personality_presets.keys()), "rules_loaded": True})


# 기본 점수화 함수들

In [ ]:
def sigmoid(x: float) -> float:
    return 1.0 / (1.0 + math.exp(-x))

def count_hits(text: str, keywords):
    return sum(1 for w in keywords if w in text)

def is_respectful_korean(text: str) -> float:
    # 존댓말 비율 근사치: '요', '습니다' 빈도 기반
    toks = re.split(r"\s+", text)
    if not toks:
        return 0.0
    hits = sum(1 for t in toks if ("요" in t or "습니다" in t))
    return hits / max(1, len(toks))

def repetition_penalty(text: str, n: int = 3) -> float:
    # 간단 n-gram 반복률 측정(근사)
    words = re.findall(r"\w+|[가-힣]+", text)
    if len(words) < n:
        return 0.0
    grams = [" ".join(words[i:i+n]) for i in range(len(words)-n+1)]
    total = len(grams)
    unique = len(set(grams))
    dup_ratio = 1 - unique/max(1,total)
    return dup_ratio  # 0~1, 높을수록 반복 많음


# 호감도 점수 산출 로직

In [ ]:
def base_sentiment_score(text: str, rule_dict: dict) -> (float, list):
    pos_c = count_hits(text, rule_dict["positive"])
    neg_c = count_hits(text, rule_dict["negative"])
    prof_c = count_hits(text, rule_dict["profanity"])

    # 기본 감정 합산
    raw = pos_c*0.6 - neg_c*0.6 - prof_c*2.0

    # 존댓말/느낌표/반복 등 가벼운 보정
    form_ratio = is_respectful_korean(text)
    raw += (form_ratio - 0.5) * 0.6  # 존댓말 비율이 0.5보다 높으면 가점

    exclaims = text.count("!")
    if exclaims > 1:
        raw -= 0.2*(exclaims-1)

    dup = repetition_penalty(text, n=3)
    if dup > 0.15:
        raw -= 0.3

    reasons = [
        {"rule":"positive_words","count":pos_c,"weight":0.6},
        {"rule":"negative_words","count":neg_c,"weight":-0.6},
        {"rule":"profanity","count":prof_c,"weight":-2.0},
        {"rule":"formality_ratio","value":round(form_ratio,2),"weight":0.6},
        {"rule":"exclaim_count","value":exclaims,"penalty_per_extra":-0.2},
        {"rule":"repetition_3gram","value":round(dup,2),"threshold":0.15,"penalty":-0.3},
    ]
    return raw, reasons

def personality_bonus(text: str, persona: dict, rule_dict: dict) -> (float, list):
    bonus = 0.0
    logs = []

    # 형식 일치
    form_ratio = is_respectful_korean(text)
    if persona.get("formality",0) > 0.7 and form_ratio > 0.8:
        bonus += 0.3
        logs.append({"rule":"formality_match","delta":0.3})

    # 직설성: 의사 표현 키워드 간단 체크
    direct_markers = ["말해볼게요","전하고 싶","원합니다","허락","바랍니다","고백"]
    if persona.get("directness",0) > 0.7 and any(m in text for m in direct_markers):
        bonus += 0.2
        logs.append({"rule":"directness_match","delta":0.2})

    # 비유 선호
    if persona.get("metaphor",0) > 0.6 and count_hits(text, rule_dict["metaphor"]) >= 1:
        bonus += 0.2
        logs.append({"rule":"metaphor_hit","delta":0.2})

    # 유머 선호
    if persona.get("humor",0) > 0.6 and count_hits(text, rule_dict["humor"]) >= 1:
        bonus += 0.2
        logs.append({"rule":"humor_hit","delta":0.2})

    # 진정성: 과장어휘 과다 시 페널티(진정성 높을수록 과장 싫어함)
    over_claims = ["최고","영원","완벽","운명","세상에 하나"]
    if persona.get("sincerity",0) > 0.7 and sum(1 for w in over_claims if w in text) >= 2:
        bonus -= 0.2
        logs.append({"rule":"overclaim_penalty","delta":-0.2})

    return bonus, logs

def affinity_score(text: str, persona_name: str) -> dict:
    persona = personality_presets.get(persona_name, list(personality_presets.values())[0])
    base_raw, base_logs = base_sentiment_score(text, reason_rules)
    p_bonus, p_logs = personality_bonus(text, persona, reason_rules)

    raw = base_raw + p_bonus
    score = sigmoid(raw)  # 0~1
    reasons = base_logs + p_logs + [{"rule":"total_raw","value":round(raw,3)}]
    return {
        "score_0_1": round(float(score), 4),
        "raw": round(float(raw), 3),
        "persona": {"name": persona_name, "vector": persona},
        "reasons": reasons
    }


# 샘플 입력과 실행

In [ ]:
samples = [
    {
        "text": "민수님, 발표를 앞두고 긴장하셨겠지만 저는 곁에서 조용히 응원하고 싶습니다. 동기로 지내며 알게 된 고마움이 커졌습니다. 조심스럽지만 제 마음을 말해볼게요.",
        "persona":"정중·낭만"
    },
    {
        "text": "민수님, 너무 과장하진 않을게요. 그냥 같이 커피 한 잔 하면서 가볍게 이야기 나눌 수 있을까요?",
        "persona":"재치·가벼움"
    },
    {
        "text": "민수님, 솔직하게 말할게요. 저는 더 가까워지고 싶습니다. 부담되지 않는 선에서 시간을 허락해 주실 수 있을까요!",
        "persona":"직설·건조"
    }
]

rows = []
for s in samples:
    out = affinity_score(s["text"], s["persona"])
    rows.append({
        "persona": s["persona"],
        "score": out["score_0_1"],
        "raw": out["raw"],
        "text": s["text"][:60] + ("..." if len(s["text"])>60 else "")
    })

df = pd.DataFrame(rows).sort_values("score", ascending=False)
df


# 단일 입력용 유틸(노트북에서 바로 써보기)

In [ ]:
def score_one(text: str, persona_name: str = "정중·낭만", verbose: bool = True):
    out = affinity_score(text, persona_name)
    if verbose:
        print(f"[persona] {persona_name}")
        print(f"[score] {out['score_0_1']} (raw={out['raw']})")
        print("[reasons]")
        for r in out["reasons"]:
            print(" -", r)
    return out

# 예시 실행
_ = score_one("민수님, 떨리는 마음이지만 진심으로 더 자주 함께 걷고 싶습니다.", "정중·낭만")


# 결과 저장(샘플/캐시)

In [ ]:
cache = {
    "samples": samples,
    "results": [affinity_score(s["text"], s["persona"]) for s in samples]
}
(DATA_DIR/"scoring_cache.json").write_text(json.dumps(cache, ensure_ascii=False, indent=2), encoding="utf-8")
print("saved:", str(DATA_DIR/"scoring_cache.json"))


# 검증 셀(노트북 품질 체크)

In [ ]:
# 간단 품질 기준: 0<=score<=1, reasons>=2, profanity=0
def profanity_hits(text: str) -> int:
    return count_hits(text, reason_rules["profanity"])

ok_range = all(0.0 <= affinity_score(s["text"], s["persona"])["score_0_1"] <= 1.0 for s in samples)
ok_reasons = all(len(affinity_score(s["text"], s["persona"])["reasons"]) >= 2 for s in samples)
ok_prof = all(profanity_hits(s["text"]) == 0 for s in samples)

print({"score_range_ok": ok_range, "reasons_len_ok": ok_reasons, "no_profanity": ok_prof})
